In [19]:
import vtk
import numpy as np

mbfvolume_path = "/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/07.Test/MBF_Territories.vtu"
mbf_path = "/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/07.Test/MBF.vtk"
from utilities import ReadVTUFile, ReadVTKFile, WriteVTUFile

In [11]:
MBF_volume = ReadVTUFile(mbfvolume_path)
MBF = ReadVTKFile(mbf_path)

In [12]:
print(dir(MBF))

['ALL_PIECES_EXTENT', 'AddObserver', 'AllocateCellGhostArray', 'AllocatePointGhostArray', 'AllocateScalars', 'AttributeTypes', 'BOUNDING_BOX', 'BreakOnError', 'CELL', 'CELL_DATA_FIELD', 'CELL_DATA_VECTOR', 'CheckAttributes', 'ComputeBounds', 'ComputeCellId', 'ComputeIndexToPhysicalMatrix', 'ComputeInternalExtent', 'ComputePointId', 'ComputeStructuredCoordinates', 'CopyAndCastFrom', 'CopyAttributes', 'CopyInformationFromPipeline', 'CopyInformationToPipeline', 'CopyStructure', 'Crop', 'DATA_EXTENT', 'DATA_EXTENT_TYPE', 'DATA_NUMBER_OF_GHOST_LEVELS', 'DATA_NUMBER_OF_PIECES', 'DATA_OBJECT', 'DATA_OBJECT_FIELD', 'DATA_PIECE_NUMBER', 'DATA_TIME_STEP', 'DATA_TYPE_NAME', 'DIRECTION', 'DataHasBeenGenerated', 'DebugOff', 'DebugOn', 'DeepCopy', 'EDGE', 'EDGE_DATA_VECTOR', 'ExtendedNew', 'FIELD', 'FIELD_ACTIVE_ATTRIBUTE', 'FIELD_ARRAY_TYPE', 'FIELD_ASSOCIATION', 'FIELD_ASSOCIATION_CELLS', 'FIELD_ASSOCIATION_EDGES', 'FIELD_ASSOCIATION_NONE', 'FIELD_ASSOCIATION_POINTS', 'FIELD_ASSOCIATION_POINTS_THE

In [15]:
print(MBF.GetNumberOfCells())
print(MBF.GetNumberOfPoints())
print(MBF.GetSpacing())

13317171
13631488
(0.390625, 0.390625, 2.0)


In [17]:
ThresholdFilter = vtk.vtkThreshold()
ThresholdFilter.SetInputData(MBF)
ThresholdFilter.SetUpperThreshold(0)
ThresholdFilter.SetThresholdFunction(vtk.vtkThreshold.THRESHOLD_UPPER)
ThresholdFilter.SetInputArrayToProcess(0,0,0,"vtkDataObject::FIELD_ASSOCIATION_POINTS","ImageScalars")
ThresholdFilter.Update()
MBF_Vol = ThresholdFilter.GetOutput()

In [20]:
WriteVTUFile("/Users/ana/Documents/AnahitaSeresti/03_MBFValidation/07.Test/MBF_Territories_new.vtu", MBF_Vol)

In [ ]:
NCells = MBF.GetNumberOfCells()
print(MBF.GetCell(0))
print(dir(MBF.GetCell(0)))
print(dir(MBF.GetCell(0).GetPoints()))
print(MBF.GetCell(0).GetPointIds())

#print(MBF.GetCell(0).GetPoints().GetPointData().GetArray("ImageScalars"))

vtkVoxel (0x7fde7bc3dfd0)
  Debug: Off
  Modified Time: 917
  Reference Count: 2
  Registered Events: (none)
  Number Of Points: 8
  Bounds: 
    Xmin,Xmax: (-78.8047, -78.4141)
    Ymin,Ymax: (-290.305, -289.914)
    Zmin,Zmax: (-246.5, -244.5)
    Point ids are: 0, 1, 512, 513, 262144, 262145, 262656, 262657  
  Merge Tolerance: 0.01
  Line:
None
  Pixel:
None


['AddObserver', 'BreakOnError', 'CellBoundary', 'Clip', 'ComputeBoundingSphere', 'ComputeCentroid', 'Contour', 'DebugOff', 'DebugOn', 'DeepCopy', 'Derivatives', 'EvaluateLocation', 'EvaluatePosition', 'FastDelete', 'GetAddressAsString', 'GetBounds', 'GetCellDimension', 'GetCellType', 'GetCentroid', 'GetClassName', 'GetCommand', 'GetDebug', 'GetEdge', 'GetEdgeArray', 'GetEdgePoints', 'GetEdgeToAdjacentFaces', 'GetEdgeToAdjacentFacesArray', 'GetFace', 'GetFaceArray', 'GetFacePoints', 'GetFaceToAdjacentFaces', 'GetFaceToAdjacentFacesArray', 'GetFaces', 'GetGlobalWarningDisplay', 'GetIsInMemkind', 'GetLength2', 'GetMTime', 'GetMe

In [37]:
ImageScalars = MBF_Vol.GetPointData().GetArray("ImageScalars")
voxel_ = MBF_Vol.GetCell(0)
id_list = voxel_.GetPointIds()

for i in range(id_list.GetNumberOfIds()):
    id_ = id_list.GetId(i)
    mbf_at_point = ImageScalars.GetValue(id_)
    print(id_,": ", mbf_at_point)

0 :  152
1 :  152
2 :  152
3 :  154
4 :  112
5 :  114
6 :  113
7 :  114


In [49]:
cbounds = MBF_Vol.GetCell(0).GetBounds()
#print(MBF_Vol.GetCell(0))
#print(cbounds[0])
cvolume = abs(cbounds[0] - cbounds[1]) * abs(cbounds[2] - cbounds[3]) * abs(cbounds[4] - cbounds[5])
print(cvolume)

0.30517578125


In [51]:
ImageScalars = MBF_Vol.GetPointData().GetArray("ImageScalars")
voxel_ = MBF_Vol.GetCell(0)
id_list = voxel_.GetPointIds()
cell_bounds = voxel_.GetBounds()
cell_volume = abs(cell_bounds[0] - cell_bounds[1]) * abs(cell_bounds[2] - cell_bounds[3]) * abs(cell_bounds[4] - cell_bounds[5])

average_cell_mbf = 0
for i in range(id_list.GetNumberOfIds()):
    id_ = id_list.GetId(i)
    mbf_at_point = ImageScalars.GetValue(id_)
    average_cell_mbf += mbf_at_point

cell_flow = average_cell_mbf/id_list.GetNumberOfIds()*cell_volume
print(cell_flow)

40.55023193359375


In [53]:
NCells = MBF_Vol.GetNumberOfCells()
Flow = 0
for n in range(NCells):
    voxel_ = MBF_Vol.GetCell(n)
    id_list = voxel_.GetPointIds()
    cell_bounds = voxel_.GetBounds()
    cell_volume = abs(cell_bounds[0] - cell_bounds[1]) * abs(cell_bounds[2] - cell_bounds[3]) * abs(cell_bounds[4] - cell_bounds[5])

    average_cell_mbf = 0
    for i in range(id_list.GetNumberOfIds()):
        id_ = id_list.GetId(i)
        mbf_at_point = ImageScalars.GetValue(id_)
        average_cell_mbf += mbf_at_point

    cell_flow = average_cell_mbf/id_list.GetNumberOfIds()*cell_volume
    Flow += cell_flow/1000
print(Flow/100)

127.64187860431994
